In [7]:
from collections import defaultdict, OrderedDict
from datetime import datetime

import numpy as np
import pymongo
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering

In [8]:


class Settings:
    MONGODB_SERVER = "localhost"
    MONGODB_PORT = 27017
    MONGODB_DB = "beautiful_creature"
    MONGODB_INVESTOR_COLLECTION = "investors"
    MONGODB_PORTFOLIO_COLLECTION = "portfolios"

    def __getitem__(self, key):
        return getattr(self, key)


timestamp = datetime.now().strftime("%d-%m-%y")
settings = Settings()
connection = pymongo.MongoClient(
    settings['MONGODB_SERVER'],
    settings['MONGODB_PORT']
)
investor_collection_name = f"{settings['MONGODB_INVESTOR_COLLECTION']}_{timestamp}"
portfolio_collection_name = f"{settings['MONGODB_PORTFOLIO_COLLECTION']}_{timestamp}"
db = connection[settings['MONGODB_DB']]
investor_collection = db[investor_collection_name]
portfolio_collection = db[portfolio_collection_name]
investors = list(investor_collection.find({}))
investors = {i["UserName"]: i for i in investors}

In [9]:
portfols = list(portfolio_collection.find({}))

In [10]:
items = defaultdict(list)

In [11]:
for p in portfols:
    for i in p["items"]:
        if i["type"] == "Buying":
            try:
                items[i["company_ticker"]].append(round(float(i["invested"].replace("%", "")), 3))
            except ValueError as e:
                if "<0.01" in str(e):
                    pass
                else:
                    raise(e)

In [12]:
# basic portfolio settings
TOP_N_BY_COUNTS = 100
TOP_N_BY_RATIO = 30

# weighted portfolio settings
TOP_N_BY_VALUE = 30
N_CLUSTERS = 10
TOP_INVESTORS_WITHIN_CLUSTER = 2

In [13]:
stats = []
for item in items:
    item_stat = {"item_name": item, "count":len(items[item]),
                 "mean": np.mean(items[item]), "std": np.std(items[item])}
    stats.append(item_stat)

df = pd.DataFrame.from_records(stats)
df["std/mean"] = df["std"] / df["mean"]
df = df.set_index("item_name")
df = df.sort_values("count",ascending=False).head(TOP_N_BY_COUNTS)

In [14]:
portfol_basic = df.sort_values("count",ascending=False).sort_values("std/mean").head(TOP_N_BY_RATIO)
portfol_basic["portfol_val"] = portfol_basic["mean"] * 100 / portfol_basic["mean"].sum()
portfol_basic.sort_values("portfol_val",ascending=False)

,count,mean,std,std/mean,portfol_val
item_name,,,,,
AMD,65,2.593231,1.662106,0.640940,4.956461
1810.HK,37,2.464054,1.506671,0.611460,4.709565
ORSTED.CO,24,2.353333,1.532910,0.651378,4.497943
PDD,25,2.286800,1.344847,0.588091,4.370778
CSIQ,44,2.240682,1.284837,0.573413,4.282632
DOYU,35,2.235143,1.463445,0.654743,4.272045
NIO,50,2.131000,1.396377,0.655268,4.072996
TSLA,47,2.097021,1.460657,0.696539,4.008052
BYND,38,2.076579,1.326811,0.638941,3.968981


In [15]:
a_portfols = []
for p in portfols:
    for i in p["items"]:
        if i["type"] == "Buying":
            try:
                a_portfols.append((p["investor_name"], float(i["value"].replace("%","")), i["company_ticker"]))
            except ValueError as e:
                if "<0.01" in str(e):
                    pass
                else:
                    raise(e)
a_portfols

[('JeppeKirkBonde', 4.25, 'CNC'),
 ('JeppeKirkBonde', 3.79, 'NDA.DE'),
 ('JeppeKirkBonde', 3.7, 'AMZN'),
 ('JeppeKirkBonde', 4.0, 'NTDOY'),
 ('JeppeKirkBonde', 3.21, 'TLRY'),
 ('JeppeKirkBonde', 3.47, 'LDOS'),
 ('JeppeKirkBonde', 2.63, 'NBIX'),
 ('JeppeKirkBonde', 3.52, 'FB'),
 ('JeppeKirkBonde', 3.25, 'NTES'),
 ('JeppeKirkBonde', 2.44, '00257.HK'),
 ('JeppeKirkBonde', 4.41, 'GOOG'),
 ('JeppeKirkBonde', 2.56, 'FSLR'),
 ('JeppeKirkBonde', 2.09, 'SOBI.ST'),
 ('JeppeKirkBonde', 2.31, 'FXPO.L'),
 ('JeppeKirkBonde', 1.68, 'VRTX'),
 ('JeppeKirkBonde', 2.87, 'UBSG.ZU'),
 ('JeppeKirkBonde', 2.99, 'INDA'),
 ('JeppeKirkBonde', 1.54, 'ZEPP'),
 ('JeppeKirkBonde', 2.48, 'NVTKL.L'),
 ('JeppeKirkBonde', 2.01, 'SMSN.L'),
 ('JeppeKirkBonde', 1.56, 'CRSR'),
 ('JeppeKirkBonde', 2.48, 'BLK'),
 ('JeppeKirkBonde', 1.99, 'AAPL'),
 ('JeppeKirkBonde', 2.07, 'HUSQB.ST'),
 ('JeppeKirkBonde', 1.51, 'WY'),
 ('JeppeKirkBonde', 1.44, 'TRIG.L'),
 ('JeppeKirkBonde', 2.85, 'CDI.PA'),
 ('JeppeKirkBonde', 1.31, 'INTC'),


In [16]:
a_df = pd.DataFrame(data=a_portfols,columns=["investor","percent","ticker"])
a_df = a_df.pivot_table(index="investor", columns="ticker").fillna(0)
a_df["cluster"] = KMeans(n_clusters=N_CLUSTERS).fit_predict(a_df)
fields = ["Copiers","WeeksSinceRegistration","DailyDD","WeeklyDD","RiskScore","Gain"]

for investor_name in a_df.index:
    for field in fields:
        a_df.loc[investor_name,field] = investors[investor_name][field]
a_df_short = a_df.drop("percent",axis=1)
a_df_short

,cluster,Copiers,WeeksSinceRegistration,DailyDD,WeeklyDD,RiskScore,Gain
ticker,,,,,,,
investor,,,,,,,
2BSmart,4,323.0,226.0,-4.38,-7.63,4.0,37.65
ABDUCT,1,584.0,95.0,-3.12,-3.12,3.0,26.57
ALnayef,1,1708.0,263.0,-2.21,-3.19,3.0,15.60
Abbroush,1,514.0,63.0,-1.32,-1.93,3.0,16.17
Aguero1010,1,1101.0,63.0,-4.26,-5.58,4.0,80.07
...,...,...,...,...,...,...,...
tradefx525,1,656.0,174.0,-2.02,-3.21,3.0,30.15
vidinho,1,133.0,627.0,-3.51,-4.45,4.0,27.47


In [17]:
top_investors = a_df_short.groupby("cluster")["Gain"].nlargest(TOP_INVESTORS_WITHIN_CLUSTER).reset_index()["investor"]
a_df_short.loc[top_investors]

,cluster,Copiers,WeeksSinceRegistration,DailyDD,WeeklyDD,RiskScore,Gain
ticker,,,,,,,
investor,,,,,,,
Raschnyr,0,198.0,500.0,-4.17,-8.87,5.0,62.66
Lvnce6,0,246.0,188.0,-4.80,-8.58,5.0,46.75
Isbelle,1,643.0,212.0,-3.72,-7.69,4.0,177.72
VidovM,1,214.0,69.0,-3.82,-5.03,4.0,114.65
myhungetoro,2,388.0,98.0,-2.09,-3.29,3.0,5.48
georgech89,3,653.0,117.0,-3.91,-6.20,5.0,53.28
sgstjc,4,1440.0,85.0,-4.88,-8.51,4.0,146.61
CPHequities,4,19258.0,223.0,-4.52,-11.12,4.0,92.29


In [18]:
portfol_weight = a_df.loc[top_investors]["percent"].sum(axis=0)
portfol_weight = portfol_weight[portfol_weight!=0]
portfol_weight = portfol_weight*100/portfol_weight.sum()
portfol_weight = portfol_weight.sort_values(ascending=False)[:TOP_N_BY_VALUE]
portfol_weight = portfol_weight*100/portfol_weight.sum()
portfol_weight

ticker
TLT        14.759093
SVXY        8.159602
FB          7.351733
AMZN        6.193661
SHV         6.151940
VTI         5.660139
BABA        4.703086
IVV         4.609530
MU          2.747260
DBX         2.658761
INTC        2.394528
WDC         2.373036
TWTR        2.333843
UPRO        2.313615
QQQ         2.049382
SOXX        2.039268
VNQ         2.019040
SQ          1.963412
PSH.L       1.952034
ITOT        1.928012
MSFT        1.840778
IEF         1.835721
AMD         1.799057
GLD         1.700444
9988.HK     1.585395
NET         1.533560
TTE.PA      1.522182
FTNT        1.312313
AIR.PA      1.268063
GOOG        1.241514
dtype: float64

In [19]:
ticker_to_name_mapping = defaultdict(str)
for p in portfols:
    for i in p["items"]:
        ticker_to_name_mapping[i["company_ticker"]] = i.get("company_name",None)

portfol_weight_new = portfol_weight.to_frame("portfol_val")
portfol_weight_new["company_name"] = portfol_weight_new.index.map(ticker_to_name_mapping)
portfol_weight_new = portfol_weight_new.reset_index()
portfol_weight_new

,ticker,portfol_val,company_name
0,TLT,14.759093,None
1,SVXY,8.159602,None
2,FB,7.351733,Facebook
3,AMZN,6.193661,Amazon
4,SHV,6.151940,None
5,VTI,5.660139,None
6,BABA,4.703086,Alibaba
7,IVV,4.609530,None
8,MU,2.747260,"Micron Technology, Inc."
9,DBX,2.658761,Dropbox Inc
